<a href="https://colab.research.google.com/github/SeongwonTak/RecommenderSystem/blob/main/Recommendation_with_MatrixFactorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendation_with_MatrixFactorization
https://lsjsj92.tistory.com/569?category=853217
을 기반으로 따라 실습해보았다.




## 데이터셋 준비.
영화 관련 데이터를 가져오려 한다.
평점과 제목만 있으면 된다.

In [20]:
import numpy as np
import pandas as pd
import sklearn as sk

In [21]:
movie_data_raw = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RecoExample/movies_metadata.csv')
movie_ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RecoExample/ratings_small.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
movie_data_raw.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [22]:
movie_data = movie_data_raw[['id', 'title']]
movie_data.rename(columns={"id":"movieId"}, inplace = True)
movie_data

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II
...,...,...
45461,439050,Subdue
45462,111109,Century of Birthing
45463,67758,Betrayal
45464,227506,Satan Triumphant


In [ ]:
movie_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movie_ratings.drop('timestamp', axis = 1, inplace=True)

두 데이터를 합치려고 한다.

In [23]:
movie_ratings.movieId = movie_ratings.movieId.astype('str')
movie_data.movieId = movie_data.movieId.astype('str')
user_movie_rating = pd.merge(movie_ratings, movie_data, on = 'movieId')
user_movie_rating

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,userId,movieId,rating,timestamp,title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III
...,...,...,...,...,...
44989,652,129009,4.0,1442690827,Love Is a Ball
44990,653,2103,3.0,948161066,Solaris
44991,659,167,4.0,836137550,K-PAX
44992,659,563,3.0,834694187,Starship Troopers


In [ ]:
user_movie_pivot = user_movie_rating.pivot_table('rating',
                                                 index='userId',
                                                 columns='title').fillna(0)

In [ ]:
user_movie_pivot.head()

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,13 Tzameti,1408,15 Minutes,16 Blocks,1900,1942: A Love Story,1984,2 Days in Paris,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,2046,2061 - Un anno eccezionale,21 Grams,24 Hour Party People,25th Hour,28 Days Later,28 Weeks Later,29th Street,2:37,3 Ninjas: High Noon at Mega Mountain,3-Iron,300,33 Scenes from Life,"4 Months, 3 Weeks and 2 Days",40 Days and 40 Nights,42nd Street,48 Hrs.,5 Card Stud,5 Days of War,...,World Trade Center,Wuthering Heights,X-Men Origins: Wolverine,X: The Unknown,Xuxa in Crystal Moon,Y Tu Mamá También,Yaji and Kita: The Midnight Pilgrims,Yamakasi,Yankee Doodle Dandy,Yella,Yesterday,You Are Not I,You Instead,You Only Live Twice,You're a Big Boy Now,"You, Me and Dupree",Young Adam,Young Black Stallion,Young Frankenstein,Young Mr. Lincoln,Young and Innocent,"Young, Violent, Dangerous",Z,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zatoichi,Zatôichi's Pilgrimage,Zazie dans le métro,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
movie_user_pivot = user_movie_pivot.values.T

In [ ]:
user_movie_pivot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## SVD 적용
scikit-learn에는 다행히 이미 SVD가 구현되어 있다. 이를 적용하자.

In [ ]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components = 12)
matrix = SVD.fit_transform(movie_user_pivot)
matrix.shape

(2794, 12)

In [ ]:
corr = np.corrcoef(matrix)

movie_title = user_movie_pivot.columns
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index('Toy Story')
corr_coffey_hands = corr[coffey_hands]
list(movie_title[(corr_coffey_hands >= 0.9)])[:50]

## Matrix_Factorization을 활용한 사용자 기반 추천 구현

한편으로는 Matrix Factorization을 활용하여 사용자 기반 추천 또한 구현이 가능하다.

SVD 추천을 위해 먼저 user iD별 추천한 영화 데이터를 표로 만들어야 한다. 이는 pivot화로 가능하다.

In [ ]:
# 사용 데이터 재점검
movie_ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [24]:
# Pivot 정의
df_user_movie_ratings = movie_ratings.pivot(
    index = 'userId',
    columns = 'movieId',
    values = 'rating'
).fillna(0)

In [25]:
df_user_movie_ratings.head()

movieId,1,10,100,100017,100032,100034,100083,100106,100159,100163,100226,100272,1003,100304,100306,100326,100365,100383,100390,1004,100450,100487,100498,1005,100517,100527,100553,100556,100581,1006,1007,100714,100745,1008,100843,1009,101,1010,101025,101070,...,99089,991,99106,99112,99114,99117,99145,99149,992,99220,99270,99273,99276,99296,99320,994,99415,99437,99468,99470,99574,996,99609,99615,99669,99675,997,99728,99741,99764,99795,998,99811,99813,99839,99846,999,99912,99917,99992
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


SVD를 적용하기 전 user간의 bias를 보정하기 위해 mean-centered를 활용할 것이다.

mean-centered의 적용은 간단하게, user별로 평균 평점을 빼주면 된다.

In [26]:
matrix = np.matrix(df_user_movie_ratings)
user_ratings_mean = np.mean(matrix, axis = 1)
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [ ]:
matrix

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 4., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [4., 0., 0., ..., 0., 0., 0.],
        [5., 0., 0., ..., 0., 0., 0.]])

In [27]:
pd.DataFrame(matrix_user_mean, columns = df_user_movie_ratings.columns).head()

movieId,1,10,100,100017,100032,100034,100083,100106,100159,100163,100226,100272,1003,100304,100306,100326,100365,100383,100390,1004,100450,100487,100498,1005,100517,100527,100553,100556,100581,1006,1007,100714,100745,1008,100843,1009,101,1010,101025,101070,...,99089,991,99106,99112,99114,99117,99145,99149,992,99220,99270,99273,99276,99296,99320,994,99415,99437,99468,99470,99574,996,99609,99615,99669,99675,997,99728,99741,99764,99795,998,99811,99813,99839,99846,999,99912,99917,99992
0,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,...,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625
1,-0.029230,3.970770,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,...,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230
2,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,...,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075
3,-0.097838,3.902162,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,...,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838,-0.097838
4,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128,-0.043128

U, Sigma V를 얻기 위해 Scikit-learn에서 쓰는 SVD가 아닌, **scipy의 SVD**를 사용하자.

Remark. Scikit-learn 에서는 U, sigma, V를 얻을 수 없다.

In [28]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [29]:
sigma

array([105.72437051, 110.15522471, 116.29714259, 118.66831462,
       122.82232277, 135.96585855, 138.92664501, 145.53348705,
       160.48685586, 199.93327534, 242.13488251, 462.13504865])

문제는, sigma는 대각행렬 형태로 나와야 하는데 그렇지가 않다. 이를 변형해줘야 한다.

In [30]:
sigma = np.diag(sigma)
sigma

array([[105.72437051,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        , 110.15522471,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        , 116.29714259,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , 118.66831462,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        122.82232277,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,

분해가 되었기에, 이제 다시 원본 행렬로 복구한다.
따라서 내적으로 U, sigma, Vt를 순서대로 곱해준 후 맨 마지막에 평균을 다시 더해서 복구한다.

In [31]:
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [32]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_movie_ratings.columns)
df_svd_preds

movieId,1,10,100,100017,100032,100034,100083,100106,100159,100163,100226,100272,1003,100304,100306,100326,100365,100383,100390,1004,100450,100487,100498,1005,100517,100527,100553,100556,100581,1006,1007,100714,100745,1008,100843,1009,101,1010,101025,101070,...,99089,991,99106,99112,99114,99117,99145,99149,992,99220,99270,99273,99276,99296,99320,994,99415,99437,99468,99470,99574,996,99609,99615,99669,99675,997,99728,99741,99764,99795,998,99811,99813,99839,99846,999,99912,99917,99992
0,-0.079686,0.038195,0.001680,-0.005592,-0.002767,-0.006465,-0.004923,-0.004255,0.016974,-0.003149,-0.003944,-0.005654,-0.002966,-0.003008,-0.003733,-0.003320,-0.002239,0.014000,-0.003944,-0.002539,-0.003980,-0.001432,-0.001865,-0.001970,-0.001710,-0.005813,-0.007294,-0.001680,-0.001494,-0.003527,0.005865,0.020649,-0.004644,-0.000656,-0.007823,0.030435,0.028662,0.020257,-0.001120,-0.006842,...,-0.003989,-0.002069,-0.003320,0.018452,-0.008010,-0.003030,0.015745,-0.010313,-0.003101,-0.004212,-0.004324,-0.004324,-0.004324,-0.003963,-0.005160,0.011495,-0.003632,-0.001431,-0.002239,-0.004192,-0.003008,0.005674,-0.005947,-0.005829,-0.003733,-0.005811,-0.005020,0.013889,-0.004101,-0.005891,-0.000291,0.000743,-0.000654,-0.006657,-0.003989,-0.003008,-0.011817,-0.003733,-0.005267,-0.003008
1,1.428452,1.970031,0.102076,-0.000855,-0.007758,0.006827,-0.000714,-0.014124,0.041923,-0.006341,-0.007644,0.009309,0.017677,-0.014842,0.011387,-0.012443,-0.009586,0.045632,-0.007644,-0.003082,-0.009182,-0.013696,-0.013029,0.028400,-0.011414,-0.009024,-0.008721,-0.001984,0.017031,0.023716,0.049900,0.028206,-0.008027,0.024430,0.008490,0.084949,-0.009121,0.011729,-0.016095,0.005511,...,0.021431,0.010941,-0.012443,-0.010990,-0.006218,-0.016232,0.059930,0.002721,0.034871,0.015962,0.013227,0.013227,0.013227,0.009020,-0.002866,0.068813,-0.010044,-0.014064,-0.009586,0.006654,-0.014842,0.025473,-0.001059,-0.000991,0.011387,0.010514,-0.001603,0.009292,0.018696,0.017628,-0.009233,0.011822,-0.015069,-0.014429,0.021431,-0.014842,0.043731,0.011387,0.004431,-0.014842
2,0.977246,0.383576,-0.034871,0.009758,-0.014160,0.025892,0.008485,0.029120,-0.008718,-0.001078,-0.006007,0.027542,0.037181,-0.023277,0.022892,-0.017520,-0.020515,-0.030787,-0.006007,0.011845,-0.006576,-0.028775,-0.021589,0.037359,-0.026871,0.008904,0.022276,-0.004828,-0.013233,0.025960,0.031040,-0.055612,0.006725,0.008507,0.031371,0.003060,-0.081602,-0.007464,-0.034531,0.024668,...,-0.002340,0.014919,-0.017520,-0.097877,0.248154,-0.019505,-0.016751,0.006242,0.005300,0.001060,0.002761,0.002761,0.002761,0.020566,0.013767,-0.048491,-0.011764,-0.005544,-0.020515,0.018240,-0.023277,0.029713,0.009005,0.009256,0.022892,0.024239,0.020484,-0.043984,-0.000640,0.047245,0.013120,0.030544,-0.039582,0.017324,-0.002340,-0.023277,0.002396,0.022892,0.019402,-0.023277
3,1.870844,1.461694,-0.312081,-0.023525,-0.067773,-0.014779,-0.035284,-0.082843,0.088169,-0.052320,0.034544,0.049592,0.066644,0.082823,-0.004834,0.066730,-0.081282,0.043716,0.034544,-0.030192,-0.063269,0.018850,-0.010507,0.159055,-0.094791,-0.030714,-0.010441,-0.087977,0.007213,-0.091239,0.278075,0.016024,-0.052650,0.002769,-0.027652,0.864168,0.348450,0.655178,0.034944,-0.023014,...,0.050500,-0.098385,0.066730,-0.008580,0.141698,0.031559,0.108527,-0.022663,-0.076728,0.034441,0.026412,0.026412,0.026412,-0.006614,-0.015307,-0.011794,0.050637,-0.016856,-0.081282,-0.008395,0.082823,0.029334,-0.028419,-0.026788,-0.004834,-0.035701,-0.038797,0.111271,0.042471,0.135007,0.069807,0.151305,-0.121809,-0.057573,0.050500,0.082823,-0.151102,-0.004834,0.017928,0.082823
4,1.182777,0.535018,-0.119556,-0.019525,-0.040942,-0.007986,-0.026900,-0.014092,0.050050,-0.034716,-0.030890,0.015190,-0.105109,-0.036285,-0.000941,-0.034487,-0.044804,0.028662,-0.030890,-0.048582,-0.007491,-0.036439,-0.038067,0.076574,-0.048665,-0.020211,-0.033116,0.011764,-0.025405,0.057885,-0.013005,0.018386,-0.042790,0.010335,-0.020963,-0.068695,0.081053,0.035891,-0.038238,-0.022473,...,-0.031301,-0.025257,-0.034487,-0.0490

Data 준비가 완료되었다.

이제 실제 추천 시스템을 만들고자 한다. 다음 기능을 반영한다.
- 사용자가 보지 않은 영화중에서 평점이 높은 것을 추천하자.
- 단, 위에서 사용한 SVD를 기준으로 한다.

In [33]:
def recommend_movies(df_svd_preds, user_id,
                     ori_movies_df, ori_ratings_df, num_recommendations = 5):
  # index 적용을 위하여
  user_row_number = user_id - 1
  # 최종 svd 결과물에서, Target user에서 영화 평점 높은대로 정렬
  sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending = False)
  # 원본 평점 데이터에서, user id에 해당하는 정보만 추출.
  user_data = ori_ratings_df[ori_ratings_df.userId == user_id]
  # user data와 원본 영화 데이터 합치기
  user_history = user_data.merge(ori_movies_df, on = 'movieId').sort_values(['rating'], ascending = False)
  # 원본 영화 데이터에서 사용자가 이미 본 영화는 제외시켜려고 한다.
  recommendations = ori_movies_df[~ori_movies_df['movieId'].isin(user_history['movieId'])]
  # 위의 추천 데이터와 사용자의 영화 평점 순위 데이터를 합친다.
  recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'movieId')
  # column 이름을 바꾼다.
  recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
  return user_history, recommendations

In [37]:
desired_id = int(input())
already_rated, predictions = recommend_movies(df_svd_preds, desired_id, movie_data, movie_ratings, 10)
predictions

222


,movieId,title,Predictions
532,913,The Thomas Crown Affair,0.673702
1624,920,Cars,0.670931
260,910,The Big Sleep,0.602742
1110,898,Birdman of Alcatraz,0.584944
1340,750,Murder She Said,0.581920
254,903,Cool Hand Luke,0.571406
895,1282,Dogtown and Z-Boys,0.562119
1715,1252,Lonely Hearts,0.553398
1404,919,Blood: The Last Vampire,0.550914
1536,1947,An Unfinished Life,0.509035


특정 유저 id를 입력하였을 때, 상위 10개의 추천을 얻어내는 시스템이다.